In [99]:
import numpy as np


In [100]:
#This is an attempt to execute a basic HF iteration of H2 in STO-3G
#Due to the incapability, the processes of molecular integrals are skipped. Values in Szabo,Ostlund are used.
#2024.02.26

#This calculation is for R = 1.4 a.u
S_matrix = np.array([[1.0, 0.6593],[0.6593,1.0]])
H_core = ([-1.1204,-0.9584],[-0.9584,-1.1204])

def molecular_integrals(u,v,a,b):
    #first excise the repeating one i.e u<v p<q
    if v > u:
        u, v = v , u
    if b > a:
        a, b = b, a 
    #define 4 types of integralsbonus = 0
    if u == v and a == b:
        if u == a:
            return 0.7746
        else :
            return 0.5697
    if (u == v and a != b) or ( u!= v and a ==b ):
        return 0.4441
    else:
        return 0.2970

#eg is eigenvalue while U is the eigen-vector matrix(Also the Unitary matrix U)    attention: diagonalizing doesnt mean orthonormalizing 

eg, U = np.linalg.eig(S_matrix)
s_r = np.zeros([2,2])
for k in range(len(S_matrix[0])):
    s_r[k][k] = 1/eg[k] ** 0.5
print(s_r)
X = np.dot(U,s_r)    
X_t= np.matrix.getH(X)
np.dot(X_t,np.dot(S_matrix,X))
print(np.dot(X_t,np.dot(S_matrix,X)))
print('----------------------')


[[0.77631422 0.        ]
 [0.         1.71322315]]
[[ 1.00000000e+00 -1.26606298e-17]
 [ 1.04135274e-16  1.00000000e+00]]
----------------------


In [101]:
def molecular_integrals(u,v,a,b):
    #first excise the repeating one i.e u<v p<q
    if v > u:
        u, v = v , u
    if b > a:
        a, b = b, a 
    #define 4 types of integralsbonus = 0
    if u == v and a == b:
        if u == a:
            return 0.7746
        else :
            return 0.5697
    if (u == v and a != b) or ( u!= v and a ==b ):
        return 0.4441
    else:
        return 0.2970
    
molecular_integrals(2,1,1,2)

0.297

In [102]:
import math
#Having the X, (uv|ab),S now get the F and then transfer F into F' as well as get the orthogonal basis

#initialize a density matrix P
P = np.array([[0.,0.],[0,0]])
Temp_save = 0
HF_energy = 0
Cover = False
count = 0
while not Cover : 
#F = H_core + Sum_ab P_ab*((uv|ab)-0.5*(ua|vb))
    F = np.zeros(P.shape)

    for i in range(len(F[0])):
        for j in range(len(F[0])):
            inte_item = 0
            for a in range(len(F[0])):
                for b in range(len(F[0])):
                    inte_item += P[a][b]*(molecular_integrals(i,j,a,b)-0.5*molecular_integrals(i,a,j,b))
            F[i][j] = H_core[i][j] + inte_item
    print('\n--------------------\n F in this literation\n',F,'\n-------------------------')   
    
    #F'=X^T*F*X diagonalize F' to get the e and C'
    X_t = np.matrix.getH(X)
    F_temp = np.dot(X_t,np.dot(F,X))
    print('Fock_tenp \n',F_temp)
    orb_e, C_temp = np.linalg.eig(F_temp)
    C_matrix = np.dot(X,C_temp)
    print(orb_e)
    print(C_matrix)

    #new P matrix & expectation value
         
    P_new = np.zeros(P.shape)
    for i in range(len(F[0])):
        for j in range(len(F[0])):
            for a in range(1):
                P_new[i][j]+= 2*C_matrix[i][a]*C_matrix[j][a]
    P = P_new          
    E_Temp = 0
    for i in range(len(F[0])):
        for j in range(len(F[0])):
            E_Temp += P[j][i]*(H_core[i][j]+F[i][j])
    HF_energy = 0.5*E_Temp 
    Total = HF_energy + 1/1.4
    print('HF = %f\n' % HF_energy)
    count += 1
    print('round ',count,'end\n')
    print('iteration end \n')
    if abs(Total - Temp_save) < 10e-4:
        Cover = True
        print('\nSCF Done E=%f' % Total)
    else:
        Temp_save = Total
    
   





--------------------
 F in this literation
 [[-1.1204 -0.9584]
 [-0.9584 -1.1204]] 
-------------------------
Fock_tenp 
 [[-1.25281745e+00  3.88618287e-17]
 [ 3.05233566e-17 -4.75491635e-01]]
[-1.25281745 -0.47549163]
[[ 0.54893705 -1.2114317 ]
 [ 0.54893705  1.2114317 ]]
HF = -2.505635

round  1 end

iteration end 


--------------------
 F in this literation
 [[-0.36550336 -0.59393908]
 [-0.59393908 -0.36550336]] 
-------------------------
Fock_tenp 
 [[-5.78221201e-01 -1.99040294e-19]
 [-1.34638130e-16  6.70489363e-01]]
[-0.5782212   0.67048936]
[[ 0.54893705 -1.2114317 ]
 [ 0.54893705  1.2114317 ]]
HF = -1.831039

round  2 end

iteration end 


--------------------
 F in this literation
 [[-0.36550336 -0.59393908]
 [-0.59393908 -0.36550336]] 
-------------------------
Fock_tenp 
 [[-5.78221201e-01 -1.99040294e-19]
 [-1.34638130e-16  6.70489363e-01]]
[-0.5782212   0.67048936]
[[ 0.54893705 -1.2114317 ]
 [ 0.54893705  1.2114317 ]]
HF = -1.831039

round  3 end

iteration end 


SCF 